In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import soundfile as sf
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import librosa
from tqdm import tqdm, trange
import pickle
import matplotlib.pyplot as plt

C:\Users\admin\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
rawAudioPath = Path('data/musicnet_lousy/train_data')
song_id = 1727

sequence, samplingRate = librosa.load(rawAudioPath / (str(song_id) + '.wav'), sr=16000)
sequence = torch.tensor(sequence).float()

In [3]:
sequence.shape[0]

7152954

In [4]:
labeling = pd.read_csv(f'./data/musicnet_lousy/train_labels/{song_id}.csv')

In [5]:
labeling

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,9182,90078,43,53,4.000,1.500,Dotted Quarter
1,9182,33758,42,65,4.000,0.500,Eighth
2,9182,62430,1,69,4.000,1.000,Quarter
3,9182,202206,44,41,4.000,3.500,Whole
4,9182,62430,1,81,4.000,1.000,Quarter
...,...,...,...,...,...,...,...
6575,19196894,19421150,44,29,365.000,3.000,Dotted Half
6576,19226590,19233758,1,60,365.375,0.125,Thirty Second
6577,19226590,19233758,1,48,365.375,0.125,Thirty Second
6578,19233758,19421150,1,65,365.500,2.500,Dotted Half


In [6]:
# the sampling in the label files are for the 44KHz sampling - need to convert to our 16KHz

In [7]:
labeling['start_time'] = labeling['start_time'] / 44100
labeling['end_time'] = labeling['end_time'] / 44100

In [8]:
n_notes = 128 # include silence as additional note
n_instruments = 128 # include silence as an instrument

Each vector in the latent representation spans a 10ms window from the original audio, ie. $0.01 \times 16000 = 160$ samples

In [9]:
window_size_ms = 10

In [10]:
downsampling = window_size_ms / 1000 * samplingRate

In [11]:
n_windows = int(np.ceil(sequence.shape[0] / downsampling))

In [12]:
target_matrix_tensor = torch.zeros(n_windows, n_notes + 1, n_instruments + 1)

In [13]:
for i in trange(n_windows):
    end = (i+1) * window_size_ms / 1000
    playing = labeling[(labeling['start_time'] <= end) & \
                       (end <= labeling['end_time'])][['note', 'instrument']].values
    notes, instruments = playing[:, 0], playing[:,1]
    if len(notes) == 0:
        # if silence, then instrument == 0 plays note == 0
        target_matrix_tensor[i, 0, 0] = 1
    else:
        target_matrix_tensor[i, notes, instruments] = 1

100%|██████████| 44706/44706 [01:10<00:00, 637.46it/s]


In [18]:
# check: the first 20 tensors should be silent, then we have 5 notes being playes:

for i in range(21):
    print(target_matrix_tensor[i].sum())

tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(5.)


Wrap the above into a function:

In [ ]:
def MusicTranscripter(sequence_path,
                      labels_path,
                      n_instruments=128, # TODO need to check how many there are by inspecting all files
                      n_notes=128,
                      window_size_ms=10):
    
    '''
    Provides binary target matrices denoting all instruments that play during
    for each window_size_ms window in the latent representations
    '''

    labeling = pd.read_csv(labels_path)
    # convert time columns to seconds for simplicity
    labeling['start_time'] = labeling['start_time'] / 44100
    labeling['end_time'] = labeling['end_time'] / 44100
    
    rawAudioPath = Path(sequence_path)
    sequence, samplingRate = librosa.load(sequence_path, sr=16000)
    downsampling = window_size_ms / 1000 * samplingRate
    n_windows = int(np.ceil(sequence.shape[0] / downsampling)) # allow for aggregating windows
    transcription = torch.zeros(n_windows, n_notes + 1, n_instruments + 1)
    
    for i in range(n_windows):
        
        end = (i+1) * window_size_ms / 1000
        
        playing = labeling[(labeling['start_time'] <= end) & \
                       (end <= labeling['end_time'])][['note', 'instrument']].values
        
        notes, instruments = playing[:, 0], playing[:,1]
        
        if len(notes) == 0:
            # if silence, then instrument == 0 plays note == 0
            transcription[i, 0, 0] = 1
        else:
            transcription[i, notes, instruments] = 1
    
    return transcription